# Pytorch Geometric GNN - homogen

## Import section

In [1]:
from data_gen import data_transform_split

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd

## Section for GNN with rdkit

In [3]:
data_rdkit = data_transform_split(2)

Load CSV
Split part
Begin data creation part
Build trainset


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Build testset


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Starting SMILES conversion


0it [00:00, ?it/s]